In [3]:
import pandas as pd
import plotly.express as px

In [ ]:
users = pd.read_csv("https://raw.githubusercontent.com/upravitelev/mar231f/main/data/hw_1/users.csv")
users["dt"] = pd.to_datetime(users["dt"])

auth = pd.read_csv("https://raw.githubusercontent.com/upravitelev/mar231f/main/data/hw_1/auth.csv")
auth["login_dt"] = pd.to_datetime(auth["login_dt"])

payments = pd.read_csv("https://raw.githubusercontent.com/upravitelev/mar231f/main/data/hw_1/payments.csv")
payments["pay_dt"] = pd.to_datetime(payments["pay_dt"])


In [13]:
retention = auth.merge(users[["user_id", "media_source", "dt"]], on="user_id", how="left")
retention["lifetime"] = (retention["login_dt"] - retention["dt"]).dt.days

retention_stat = retention.groupby(["media_source", "lifetime"]).agg(n_users=("user_id", "nunique")).reset_index()
retention_stat = retention_stat.sort_values(["media_source", "lifetime"])

retention_stat["total_users"] = retention_stat.groupby("media_source")["n_users"].transform(
    lambda x: x.loc[retention_stat["lifetime"] == 0].values[0]
)

retention_stat["share"] = retention_stat["n_users"] / retention_stat["total_users"]

retention_stat = retention_stat[retention_stat["lifetime"] <= 21]

fig_line = px.line(retention_stat, x="lifetime", y="share", color="media_source", title="Удержание пользователей в зависимости от канала привлечения")
fig_line.show()

retention_stat_sorted = retention_stat.sort_values(by=["lifetime", "share"], ascending=[True, False])

retention_stat_sorted["media_source"] = retention_stat_sorted["media_source"].astype(str)

fig_bar = px.bar(
    retention_stat_sorted, x="lifetime", y="share", color="media_source", title="Удержание пользователей в зависимости от канала привлечения"
)

fig_bar.update_layout(yaxis_tickformat=".0%", yaxis_title="Доля пользователей")

fig_bar.show()

In [14]:
payments = payments.merge(users[["user_id", "media_source"]], on="user_id", how="left")

payments_grouped = payments.groupby(["pay_dt", "media_source"], as_index=False).agg(gross=("purchase", "sum"))

fig_line = px.area(
    payments_grouped, x="pay_dt", y="gross", color="media_source", title="Динамика прибыли по каналам", labels={"gross": "Прибыль", "pay_dt": "Дата"}
)
fig_line.update_layout(yaxis_range=[0, None])
fig_line.show()

fig_bar = px.bar(
    payments_grouped,
    x="pay_dt",
    y="gross",
    color="media_source",
    title="Динамика прибыли по каналам (барчарт)",
    labels={"gross": "Прибыль", "pay_dt": "Дата"},
)
fig_bar.update_layout(barmode="stack", yaxis_range=[0, None])
fig_bar.show()

In [15]:
ltv = pd.merge(users, payments, on=["user_id", "media_source"], how="left")

ltv["lifetime"] = (ltv["pay_dt"] - ltv["dt"]).dt.days

ltv_stat = ltv[(ltv["lifetime"] < 7) | (ltv["lifetime"].isna())]

grouped = (
    ltv_stat.groupby("media_source")
    .agg(
        total_users=("user_id", "nunique"),
        payers=("purchase", lambda x: x.notna().sum()),
        gross=("purchase", "sum"),
        n_purchases=("purchase", lambda x: x.notna().sum()),
        CPI=("CPI", "mean"),
    )
    .reset_index()
)

total = pd.DataFrame(
    {
        "media_source": ["Total"],
        "total_users": [ltv_stat["user_id"].nunique()],
        "payers": [ltv_stat["purchase"].notna().sum()],
        "gross": [ltv_stat["purchase"].sum()],
        "n_purchases": [ltv_stat["purchase"].notna().sum()],
        "CPI": [ltv_stat["CPI"].mean()],
    }
)

ltv_stat_final = pd.concat([grouped, total], ignore_index=True)

ltv_stat_final["conversion"] = (ltv_stat_final["payers"] / ltv_stat_final["total_users"]).round(3)
ltv_stat_final["ARPU"] = (ltv_stat_final["gross"] / ltv_stat_final["total_users"]).round(3)
ltv_stat_final["ARPPU"] = (ltv_stat_final["gross"] / ltv_stat_final["payers"]).round(3)
ltv_stat_final["Av.purchases"] = (ltv_stat_final["n_purchases"] / ltv_stat_final["payers"]).round(1)
ltv_stat_final["Av.bill"] = (ltv_stat_final["gross"] / ltv_stat_final["n_purchases"]).round(1)
ltv_stat_final["CPI"] = ltv_stat_final["CPI"].round(2)


ltv_curve = pd.merge(
    ltv[ltv["lifetime"].notna()].groupby(["media_source", "lifetime"]).agg(gross=("purchase", "sum")).reset_index(),
    ltv.groupby("media_source").agg(total_users=("user_id", "nunique")).reset_index(),
    on="media_source",
    how="left",
)

ltv_curve = ltv_curve.sort_values(["media_source", "lifetime"])

ltv_curve["cum_gross"] = ltv_curve.groupby("media_source")["gross"].cumsum()
ltv_curve["cumARPU"] = ltv_curve["cum_gross"] / ltv_curve["total_users"]

fig = px.line(ltv_curve[ltv_curve["lifetime"] <= 21], x="lifetime", y="cumARPU", color="media_source", title="Кривая LTV по каналам привлечения")
fig.update_layout(yaxis=dict(rangemode="tozero"))
fig.show()


In [16]:
onboarding = pd.read_csv("https://raw.githubusercontent.com/upravitelev/mar231f/main/data/hw_1/onboarding.csv")

onboarding_stat = onboarding.groupby(["version", "step"]).agg(n_users=("user_id", "nunique")).reset_index()

first_step_users = onboarding_stat[onboarding_stat["step"] == 1][["version", "n_users"]].rename(columns={"n_users": "total_users"})
onboarding_stat = onboarding_stat.merge(first_step_users, on="version", how="left")

onboarding_stat["share"] = onboarding_stat["n_users"] / onboarding_stat["total_users"]

onboarding_stat["version"] = onboarding_stat["version"].astype(str)

onboarding_stat["text"] = (onboarding_stat["share"] * 100).round(1).astype(str) + "%"

fig = px.bar(
    onboarding_stat,
    x="step",
    y="share",
    color="version",
    barmode="group",
    text="text",
    title="Воронка шагов онбординга по версиям",
    labels={"step": "Шаг онбординга", "share": "Доля пользователей", "version": "Версия"},
)

fig.update_traces(textposition="outside")

fig.show()